In [104]:
import requests
from pymongo import MongoClient
import ast
import pandas as pd
import numpy as np


In [310]:
client = MongoClient()
db = client.meetup
groups_col=db.groups
groups_clean_extra_col = db.groups_clean_extra
groups_clean_extra_col
test_col = db.test

In [311]:
api_key='466c256e2d1d455e4d3e4861bd32b'

In [ ]:
'https://api.meetup.com/2/groups?zip=94109&offset=0&city=San+Francisco&format=json&lon=-122.419998169&photo-host=public&state=ca&page=20&radius=25.0&fields=sponsors%2Clast_event%2Cnext_event%2Cphotos%2C+short_link&lat=37.7900009155&order=id'

In [267]:
def save_groups_raw(api_key,location,radius,offset):
    groups = requests.get('https://api.meetup.com/find/groups?key='+api_key+'&sign=true&photo-host=public&location='+location+\
                          '&radius='+str(radius)+'&page=200&offset='+str(offset))
    groups_info = ast.literal_eval(groups.text)
    for group in groups_info:
        group['cleaned']=False
        group['member_profiles']=[]
        group['description']=clean_description(group['description'])
        inserted_group = groups_col.insert_one(group)
        print inserted_group.inserted_id

        

In [ ]:
def save_groups_clean(api_key,location,radius,offset):
    groups = requests.get('https://api.meetup.com/find/groups?key='+api_key+'&sign=true&photo-host=public&location='+location+\
                          '&radius='+str(radius)+'&page=200&offset='+str(offset))
    groups_info = ast.literal_eval(groups.text)
    for group in groups_info:
        group['cleaned']=True
        group['member_profiles']=[]
        group['events']=[]
        try:
            group['next_event_id']=group['next_event']['id']
            group['next_event_name']=group['next_event']['name']
            group['next_event_time']=group['next_event']['time']
            group['next_event_offset']=group['next_event']['utc_offset']
            group['next_event_rsvp']=group['next_event']['yes_rsvp_count']
            group['organizer_id'] = group['organizer']['id']
            group['organizer_name'] = group['organizer']['name']
        except KeyError:
            group['next_event_id']=np.nan
            group['next_event_name']=np.nan
            group['next_event_time']=np.nan
            group['next_event_offset']=np.nan
            group['next_event_rsvp']=np.nan
            group['organizer_id'] = np.nan
            group['organizer_name'] = np.nan
        group['description']=clean_description(group['description'])
        inserted_group = groups_clean_col.insert_one(group)
        print inserted_group.inserted_id

In [302]:
def test(api_key,location,radius,offset,zip_code):
    groups = requests.get('https://api.meetup.com/2/groups?key='+api_key+'&sign=true&photo-host=public&zip='+str(zip_code)+'&fields=sponsors,last_event,next_event,photos, short_link&city='+location+'&state=CA&page=20&offset='+str(offset))
    return groups

groups_test=test(api_key,'San Francisco',20,0,94109)

In [330]:
group_lst=ast.literal_eval(groups_test.text.replace('\\/','/'))
group_lst['results'][10]['description']


"Our purpose is to promote social welfare and restore citizens' ownership of our democracy by mobilizing grassroots participation in the community. San Francisco for Democracy works to ensure that elected officials are held accountable to the electorate."

In [348]:
def save_groups_clean_extra_info(api_key,location,radius,offset,zip_code,state):
    groups = requests.get('https://api.meetup.com/2/groups?key='+api_key+'&sign=true&photo-host=public&zip='+str(zip_code)+'&fields=sponsors,last_event,next_event,photos, short_link,description&city='+location+'&state='+state+'&page=100&offset='+str(offset))
    groups_info = ast.literal_eval(groups.text.replace('\\/','/'))
    for group in groups_info['results']:
        #print group
        group['cleaned']=True
        group['member_profiles']=[]
        group['events']=[]
        #print group['description']
        try:
            group['next_event_id']=group['next_event']['id']
            group['next_event_name']=group['next_event']['name']
            group['next_event_time']=group['next_event']['time']
            group['next_event_offset']=group['next_event']['utc_offset']
            group['next_event_rsvp']=group['next_event']['yes_rsvp_count']
            group['organizer_id'] = group['organizer']['id']
            group['organizer_name'] = group['organizer']['name']
            #group['description']=clean_description(group['description'])
        except KeyError:
            group['next_event_id']=np.nan
            group['next_event_name']=np.nan
            group['next_event_time']=np.nan
            group['next_event_offset']=np.nan
            group['next_event_rsvp']=np.nan
            group['organizer_id'] = np.nan
            group['organizer_name'] = np.nan
            #group['description']=np.nan
        
        inserted_group = groups_clean_extra_col.insert_one(group)
        #print inserted_group.inserted_id

In [350]:
for offset in range(88):
    save_groups_clean_extra_info(api_key,'san francisco',30,offset,94109,'CA')
    print offset

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


In [8]:
#for offset in range(36,44):
for offset in range(36,44):
    save_groups(api_key,'san francisco',30,offset)
    print offset

56f18859a8326c050e01b338
56f18859a8326c050e01b339
56f18859a8326c050e01b33a
56f18859a8326c050e01b33b
56f18859a8326c050e01b33c
56f18859a8326c050e01b33d
56f18859a8326c050e01b33e
56f18859a8326c050e01b33f
56f18859a8326c050e01b340
56f18859a8326c050e01b341
56f18859a8326c050e01b342
56f18859a8326c050e01b343
56f18859a8326c050e01b344
56f18859a8326c050e01b345
56f18859a8326c050e01b346
56f18859a8326c050e01b347
56f18859a8326c050e01b348
56f18859a8326c050e01b349
56f18859a8326c050e01b34a
56f18859a8326c050e01b34b
56f18859a8326c050e01b34c
56f1885aa8326c050e01b34d
56f1885aa8326c050e01b34e
56f1885aa8326c050e01b34f
56f1885aa8326c050e01b350
56f1885aa8326c050e01b351
56f1885aa8326c050e01b352
56f1885aa8326c050e01b353
56f1885aa8326c050e01b354
56f1885aa8326c050e01b355
56f1885aa8326c050e01b356
56f1885aa8326c050e01b357
56f1885aa8326c050e01b358
56f1885aa8326c050e01b359
56f1885aa8326c050e01b35a
56f1885aa8326c050e01b35b
56f1885aa8326c050e01b35c
56f1885aa8326c050e01b35d
56f1885aa8326c050e01b35e
56f1885aa8326c050e01b35f


In [69]:
groups=requests.get('https://api.meetup.com/find/groups?key='+api_key+'&sign=true&photo-host=public&location=san francisco&radius=30&page=200&offset=1')

In [70]:
groups

<Response [200]>

In [71]:
groups_info = ast.literal_eval(groups.text)


In [72]:
len(groups_info)

200

In [73]:
for group in groups_info:
    inserted_group = groups_col.insert_one(group)
    print inserted_group.inserted_id

56f062bda8326c0ebf6c9e66
56f062bda8326c0ebf6c9e67
56f062bda8326c0ebf6c9e68
56f062bda8326c0ebf6c9e69
56f062bda8326c0ebf6c9e6a
56f062bda8326c0ebf6c9e6b
56f062bda8326c0ebf6c9e6c
56f062bda8326c0ebf6c9e6d
56f062bda8326c0ebf6c9e6e
56f062bda8326c0ebf6c9e6f
56f062bda8326c0ebf6c9e70
56f062bda8326c0ebf6c9e71
56f062bda8326c0ebf6c9e72
56f062bda8326c0ebf6c9e73
56f062bda8326c0ebf6c9e74
56f062bda8326c0ebf6c9e75
56f062bda8326c0ebf6c9e76
56f062bda8326c0ebf6c9e77
56f062bda8326c0ebf6c9e78
56f062bda8326c0ebf6c9e79
56f062bda8326c0ebf6c9e7a
56f062bda8326c0ebf6c9e7b
56f062bda8326c0ebf6c9e7c
56f062bda8326c0ebf6c9e7d
56f062bda8326c0ebf6c9e7e
56f062bda8326c0ebf6c9e7f
56f062bda8326c0ebf6c9e80
56f062bda8326c0ebf6c9e81
56f062bda8326c0ebf6c9e82
56f062bda8326c0ebf6c9e83
56f062bda8326c0ebf6c9e84
56f062bda8326c0ebf6c9e85
56f062bda8326c0ebf6c9e86
56f062bda8326c0ebf6c9e87
56f062bda8326c0ebf6c9e88
56f062bda8326c0ebf6c9e89
56f062bda8326c0ebf6c9e8a
56f062bda8326c0ebf6c9e8b
56f062bda8326c0ebf6c9e8c
56f062bda8326c0ebf6c9e8d


In [275]:
tech_groups=groups_clean_col.find({})

In [274]:
counter=0
for i in tech_groups:
    counter+=1
print counter

8209


In [276]:
tech_groups_arr=[i for i in tech_groups]


In [135]:
tech_groups_arr[:10]

[{u'_id': ObjectId('56f061d7a8326c0ebf6c9d9e'),
  u'category': {u'id': 34,
   u'name': u'Tech',
   u'shortname': u'Tech',
   u'sort_name': u'Tech'},
  u'city': u'San Francisco',
  u'country': u'US',
  u'created': 1395096137000L,
  u'description': u'<p>Meet other developers and ops engineers using Docker.\xa0Docker is an open platform for developers and sysadmins to build, ship, and run distributed applications. Consisting of Docker Engine, a portable, lightweight runtime and packaging tool, and Docker Hub, a cloud service for sharing applications and automating workflows, Docker enables apps to be quickly assembled from components and eliminates the friction between development, QA, and production environments. As a result, IT can ship faster and run the same app, unchanged, on laptops, data center VMs, and any cloud.</p>\n<p>Learn more about Docker at\xa0<a href="http://www.docker.com/">http://www.docker.com</a></p>',
  u'group_photo': {u'highres_link': u'http://photos2.meetupstatic.c

In [277]:
groups_df = pd.DataFrame(tech_groups_arr)
groups_df.shape

(8209, 34)

In [278]:
groups_df.columns

Index([u'_id', u'category', u'city', u'cleaned', u'country', u'created',
       u'description', u'group_photo', u'id', u'join_mode', u'key_photo',
       u'lat', u'link', u'localized_country_name', u'lon', u'member_profiles',
       u'members', u'name', u'next_event', u'next_event_id',
       u'next_event_name', u'next_event_offset', u'next_event_rsvp',
       u'next_event_time', u'organizer', u'organizer_id', u'organizer_name',
       u'photos', u'score', u'state', u'timezone', u'urlname', u'visibility',
       u'who'],
      dtype='object')

In [279]:
#sanfran_tech=groups_df[groups_df.category=={u'shortname': u'Tech', u'id': 34, u'name': u'Tech', u'sort_name': u'Tech'}]
groups_df.category.describe()

count                                                  8020
unique                                                   32
top       {u'shortname': u'Tech', u'id': 34, u'name': u'...
freq                                                   1610
Name: category, dtype: object

In [149]:
def get_category_name(category):
    try:
        return category['name']
    except TypeError:
        return np.nan
    
def get_category_id(category):
    try:
        return category['id']
    except TypeError:
        return np.nan
        


In [150]:
groups_df['category_name'] = groups_df.category.map(get_category_name)
groups_df['category_id']=groups_df.category.map(get_category_id)


count                                                  9000
unique                                                   32
top       {u'shortname': u'Tech', u'id': 34, u'name': u'...
freq                                                   1776
Name: category, dtype: object

In [210]:
groups_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9198 entries, 0 to 9197
Data columns (total 34 columns):
_id                       9198 non-null object
category                  9000 non-null object
city                      9198 non-null object
country                   9198 non-null object
created                   9198 non-null int64
description               9198 non-null object
group_photo               5587 non-null object
id                        9198 non-null int64
join_mode                 9198 non-null object
key_photo                 85 non-null object
lat                       9198 non-null float64
link                      9198 non-null object
localized_country_name    9198 non-null object
lon                       9198 non-null float64
members                   9198 non-null int64
name                      9198 non-null object
next_event                3583 non-null object
organizer                 9198 non-null object
photos                    5996 non-null object
sco

In [196]:
def get_next_event_id(event):
    try:
        return event['id']
    except TypeError:
        return np.nan

def get_next_event_name(event):
    try:
        return event['name']
    except TypeError:
        return np.nan

def get_next_event_time(event):
    try:
        return event['time']
    except TypeError:
        return np.nan
    
def get_next_event_offset(event):
    try:
        return event['utc_offset']
    except TypeError:
        return np.nan
    
def get_next_event_rsvp(event):
    try:
        return event['yes_rsvp_count']
    except TypeError:
        return np.nan 

def get_organizer_id(organizer):
    try:
        return organizer['id']
    except TypeError:
        return np.nan

def get_organizer_name(organizer):
    try:
        return organizer['name']
    except TypeError:
        return np.nan


In [209]:
groups_df['next_event_id']=map(get_next_event_id,groups_df.next_event)
groups_df['next_event_name']=map(get_next_event_name,groups_df.next_event)
groups_df['next_event_time']=map(get_next_event_time,groups_df.next_event)
groups_df['next_event_offset']=map(get_next_event_offset,groups_df.next_event)
groups_df['next_event_rsvp']=map(get_next_event_rsvp,groups_df.next_event)
groups_df['organizer_id']=map(get_organizer_id,groups_df.organizer)
groups_df['organizer_name']=map(get_organizer_name,groups_df.organizer)

In [225]:
groups_df.description[4]

u'<p>This group is about fun and interesting activities for thoughtful, emotionally healthy people. It\'s intended primarily for current or past participants in Divorce and Relationship Recovery groups in the San Francisco Bay Area, but is open to anyone willing to treat others with respect and compassion. <br></p>\n<p>Our events offer a way for people to re-enter the social world in a safe setting, to meet new people, and to do things together that are&nbsp;<i><b>not</b></i> focused on alcohol or romance. We insist on thoughtful, respectful behavior from everyone in the group and at our events. <br> </p>\n<p>See our <b><a href="http://www.meetup.com/Bay-Area-New-Beginnings/#calendar">event calendar</a></b>. See <b><a href="http://www.meetup.com/Bay-Area-New-Beginnings/photos/all_albums/?sort=create">photos from our recent events</a><a href="http://www.meetup.com/Bay-Area-New-Beginnings/photos"></a></b>. <br> </p>\n<p><b><a href="http://www.meetup.com/Bay-Area-New-Beginnings/tellfriend

In [241]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_backspaces(text):
    split_text = text.split()
    for i in split_text:
        i.replace('\n',' ')
    return ' '.join(split_text)

def clean_description(descr):
    no_tags = strip_tags(groups_df.description[199])
    return remove_backspaces(no_tags)

In [243]:
groups_df.description = groups_df.description.apply(clean_description)

In [245]:
groups_df.description[10]

u"Bay Area Tennis players are ready to have fun and stay fit in free events from Sunnyvale, Mountain View, Santa Clara Palo Alto, Menlo Park, San Francisco, Oakland and more locations. We have room for everyone who wants to play tennis, from beginners to advanced players. The Bay Area tennis group is the largest and most active tennis group, with hundreds of RSVPs every month. We have 3300+ members with events most days of the week, in various places around the Bay Area. Meetups are relaxed and easy going, most of the times, singles/doubles is accepted based on user preference. If you are interested in increased skill through match play, flexible tennis leagues may be suitable for you too. We partnered with RacketLogger for flex tennis leagues (ask for the meetup group discount). Always read the event details to make sure your level and interest matches the event. When you join our group, please indicate the following in your profile: 1) Your level (beginner, intermediate, advanced) 2)